In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np
import os
from reload_recursive import reload_recursive
import helpers
import pyperclip
import statsmodels.api as sm
import subprocess

from mri_data import file_manager as fm
from mri_data import utils

In [2]:
drive_root = fm.get_drive_root()
msmri_home = Path("/home/srs-9/Projects/ms_mri")
msmri_datadir = msmri_home / "data"
curr_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1")
data_dir = curr_dir / "data0"
showfigs = False
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.precision = 3

In [8]:
df1 = pd.read_csv(data_dir / "t1_data_full.csv")
df1.set_index('subid', inplace=True)
df1 = helpers.set_dz_type5(df1)

df2 = pd.read_csv(data_dir / "t1_2_data_full.csv")
df2.set_index('subid', inplace=True)
df2 = helpers.set_dz_type5(df2)

df3 = pd.read_csv(data_dir / "t1_3_data_full.csv")
df3.set_index('subid', inplace=True)
df3 = helpers.set_dz_type5(df3)

df_flair = pd.read_csv(data_dir / "flair_data_full.csv")
df_flair.set_index('subid', inplace=True)
df_flair = helpers.set_dz_type5(df_flair)

df_t1_flair = pd.read_csv(data_dir / "t1_flair_data_full.csv")
df_t1_flair.set_index('subid', inplace=True)
df_t1_flair = helpers.set_dz_type5(df_t1_flair)

In [9]:
df = pd.DataFrame(index=df1.index)
df['choroid_volume1'] = df1['choroid_volume']
df['choroid_volume2'] = df2['choroid_volume']
df['choroid_volume_fl'] = df_flair['choroid_volume']
df['choroid_volume_t1fl'] = df_t1_flair['choroid_volume']

df['cp_diff'] = df['choroid_volume1'] - df['choroid_volume2']
df['dz_type5'] = df1['dz_type5']

In [10]:
df = df.sort_values(by="choroid_volume1", ascending=False)
df_pms = df[df['dz_type5']=="PMS"]

In [25]:
sub = 1001

dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
label1 = labelroot / df1.loc[sub, 'sub-ses'] / df1.loc[sub, 'label']
label2 = labelroot / df2.loc[sub, 'sub-ses'] / df2.loc[sub, 'label']
label_flair = labelroot / df_flair.loc[sub, 'sub-ses'] / df_flair.loc[sub, 'label']
label_t1_flair = labelroot / df_t1_flair.loc[sub, 'sub-ses'] / df_t1_flair.loc[sub, 'label']

t1 = dataroot / df1.loc[sub, 'sub-ses'] / "t1.nii.gz"
t1_gd = dataroot / df1.loc[sub, 'sub-ses'] / "t1_gd.nii.gz"

flair = dataroot / df1.loc[sub, 'sub-ses'] / "flair.nii.gz"
t1_gd = dataroot / df1.loc[sub, 'sub-ses'] / "t1_gd.nii.gz"

cmd = utils.open_itksnap_workspace_cmd(images=[t1, t1_gd, flair], labels=[label1, label2, label_flair, label_t1_flair], win=False)
pyperclip.copy(cmd)
print(cmd)
subprocess.Popen(cmd.split(" "))

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1001/ses-20170215/t1.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1001/ses-20170215/t1_gd.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1001/ses-20170215/flair.nii.gz -s /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/t1_choroid_pineal_pituitary_T1-1_pred.nii.gz /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz /media/smbshare/srs-9/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


<Popen: returncode: None args: ['itksnap', '-g', '/media/smbshare/3Tpioneer_...>

In [20]:
sub = 1370
t1 = dataroot / df1.loc[sub, 'sub-ses'] / "proc/t1_std.nii.gz"
flair = dataroot / df1.loc[sub, 'sub-ses'] / "proc/flair-brain-mni_reg.nii.gz"
cmd = utils.open_itksnap_workspace_cmd(images=[t1, flair])
subprocess.Popen(cmd.split(" "))

<Popen: returncode: None args: ['itksnap', '-g', '/media/smbshare/3Tpioneer_...>

Launching ITK-SNAP


In [18]:
print(t1)

/media/smbshare/3Tpioneer_bids/sub-ms1370/ses-20210421/proc/t1_std.nii.gz


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(12, 4))

axs[0].hist(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])
axs[1].hist(df3.loc[df2['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.scatter(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'], 
            df3.loc[df3['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.hist((df1.loc[df1['dz_type5'].isin(["PMS"]), 'choroid_volume'] - df2.loc[df2['dz_type5'].isin(["PMS"]), 'choroid_volume']))

In [ ]:
model_data = df2.copy()
model_data.loc[model_data['dz_type5']=="PMS", "choroid_volume"] = df.loc[df['dz_type5']=="PMS", "choroid_volume"]

model_data = helpers.fix_edss(model_data)
model_data = helpers.clean_df(model_data)

model_data = model_data[model_data['dz_type5'].isin(["RMS", "PMS"])]
model_data = pd.concat((model_data, pd.get_dummies(model_data["sex"], dtype="int")), axis=1)

In [56]:
model_data['EDSS'] = model_data['EDSS'].astype("float")

In [57]:
res = sm.OLS.from_formula("EDSS ~ choroid_volume + tiv + age + Female", data=model_data).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   EDSS   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     23.90
Date:                Mon, 10 Mar 2025   Prob (F-statistic):           1.07e-17
Time:                        21:53:22   Log-Likelihood:                -778.40
No. Observations:                 394   AIC:                             1567.
Df Residuals:                     389   BIC:                             1587.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8740      1.428      0.